In [1]:
import psycopg2
from psycopg2.extensions import register_adapter, AsIs

In [2]:
def select(register,complexType='r',dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    if complexType=='r':
        select_stament="Sample ="+str(register['s_from'])
    else :
        select_stament="Sample BETWEEN "+str(register['s_from'])+" AND "+str(register['s_to'])
    select_stament = "SELECT sample,amp FROM "+complexType+" WHERE record='"+register['record']+"' AND "+select_stament+" LIMIT 1"
    cur.execute(select_stament)
    for row in cur :
        register[complexType+'_s']=row[0]
        register[complexType+'_a']=row[1]
    conn.close()
    return register

In [3]:
def insertRow(row,conn) :
    cur = conn.cursor()
    insert_statement = 'INSERT into rstq (%s) values %s'
    columns = row.keys()
    values = [row[column] for column in columns]
#    print(cur.mogrify(insert_statement, (AsIs(','.join(columns)), tuple(values))))
    cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))

In [4]:
def difference(first,second):
    if first is None or second is None :
        return None
    else :
        return first-second

In [5]:
def readHeartBeats(record,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT s_from,s_to FROM heartbeatstemporal WHERE s_to - s_from < 1000 AND record ='"+record+"' "
    cur.execute(select_stament)
    print(record,end=",")
    for row in cur :
        register = {'record':record,'s_from':row[0],'s_to':row[1]}
        register = select(register)
        register = select(register,complexType='s')
        register = select(register,complexType='t')
        register = select(register,complexType='q')
        register['qt'] = difference(register.get('q_s'),register.get('t_s'))
        register['ts'] = difference(register.get('t_s'),register.get('s_s'))
        register['sr'] = difference(register.get('s_s'),register.get('r_s'))
        del register['s_from']
        del register['s_to']
        insertRow(register,conn)
    conn.commit()
    cur.close()
    conn.close()
#        selectrow(record[0])

In [6]:
def distinctRecord(dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT DISTINCT r.record FROM r where record not in (select distinct record from rstq) "
    cur.execute(select_stament)
    records = []
    for record in cur :
        records.append(record[0])
    cur.close()
    conn.close()
    return records

In [7]:
records = distinctRecord()
for record in records:
    readHeartBeats(record)

mimic2wdb/matched/s04331/s04331-2696-12-22-07-41,mimic2wdb/matched/s14266/s14266-2770-01-13-15-29,mimic2wdb/matched/s14947/s14947-3433-11-06-12-07,mimic2wdb/matched/s20486/s20486-2701-07-11-18-41,mimic2wdb/matched/s04331/s04331-2696-12-18-12-06,mimic2wdb/matched/s23890/s23890-2832-10-25-12-55,mimic2wdb/matched/s15687/s15687-3453%10-21-19-11,